# Notebook to work with demand forecasting query
- 

### 

# Imports

#### Standard library imports

In [1]:
import sys
sys.path.append("../")

import os

#### Third party imports

In [2]:
import mysql.connector

#### Local application imports

In [6]:
%load_ext autoreload
%autoreload 2

from pkg_dir.config import *
from pkg_dir.src.utils import *
from pkg_dir.src.functions import *
from pkg_dir.src.parameters import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 

# Getting the data

##### Converting the mysql tables into dataframes

In [7]:
## Function parameters
db_crds = 'mysql_dp'
sql_files_path = '../pkg_dir/sql/queries/'
sql_script = 'demand_forecast_data.sql'
sql_params = sql_params_workhrs

## Function execution
sql_to_df(db_crds, sql_files_path, sql_script, sql_params)

,Inv_ID,Prod_ID,Item_condition,Size,Color,List_Price,Signed_Date,Purchase_ID,Boutique,Appraiser_ID,Consignor_ID,Commission_Rate,Discount
0,1,1,Good,M,Navy Blue,80,2023-03-25,3.0,1,1,1,0.8,NaN
1,2,1,Fair,L,Navy Blue,50,2023-04-26,NaN,2,3,2,0.5,NaN
2,3,1,Excellent,S,Pink,100,2023-02-23,4.0,3,8,3,0.8,NaN
3,4,2,Excellent,S,Green,90,2022-05-25,NaN,3,8,4,0.8,NaN
4,5,2,Fair,XL,Green,40,2022-01-12,5.0,2,4,5,0.6,NaN
5,6,2,Very Good,M,Black,60,2022-06-13,6.0,2,4,6,0.8,NaN
6,7,3,Fair,L,Black,35,2022-07-18,NaN,2,5,7,0.7,NaN
7,8,3,Fair,S,White,30,2022-05-07,7.0,2,5,8,0.5,NaN
8,9,3,Fair,XS,White,20,2023-03-02,NaN,1,1,9,0.5,0.5
9,10,4,Fair,L,White,40,2023-02-04,8.0,1,1,10,0.6,0.5


### 

# Header

### 

# *Notes*

---

---